In [1]:
import sys
sys.path.append("/scratch/arka/miniconda3/external/fastai/")
import matplotlib
matplotlib.use('Agg')

from tqdm import tqdm
tqdm.monitor_interval = 0

from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from fastai.models.senet import *

PATH = "/scratch/arka/Ark_git_files/iFood/"
sz = 224
# arch = resnext101_64
arch = inception_4
# arch = resnet50
# arch = resnext50
bs = 32

torch.cuda.is_available()

torch.backends.cudnn.enabled

files = os.listdir(f'{PATH}train_all/')
files.sort()
files[-5:-1]

train_label_csv = f'{PATH}train3_info.csv'
train2_label_csv = f'{PATH}train4_info.csv'
valid_label_csv = f'{PATH}val_info.csv'
test_label_csv = f'{PATH}test_info.csv'

train_label_df = pd.read_csv(train_label_csv, header=None)
valid_label_df = pd.read_csv(valid_label_csv, header=None)
test_label_df = pd.read_csv(test_label_csv, header=None)
train2_label_df = pd.read_csv(train2_label_csv, header=None)

train2_label_df.pivot_table(index=1, aggfunc=len).sort_values(0, ascending=True)
# len(train_label_df)

val_idxs = np.arange(len(train_label_df), len(train2_label_df))


def get_data(sz, bs):
    # tfms = tfms_from_model(arch, sz, aug_tfms=transforms_top_down, max_zoom=1.1) # -
    tfms = tfms_from_model(arch, sz, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train_all', f'{PATH}train4_info.csv',
                                        test_name='test_set', val_idxs=val_idxs,
                                        tfms=tfms, bs=bs, skip_header=False)
    return data


data = get_data(sz, bs)

# learn = ConvLearner.pretrained(arch, data, precompute=False, xtra_fc=[128])
md = se_resnext50_32x4d()
md3 = nn.Sequential(*children(md), nn.Linear(1000, 211), nn.LogSoftmax())
# data = get_data(224, 5)
learn = ConvLearner.from_model_data(md3, data)
# lrf = learn.lr_find()

# learn.sched.plot_lr()

# learn.sched.plot()


def one_to_one_map(inp_label):
    import json
    big_label_dict = json.load(open(f'{PATH}big_label_dict.json', 'r'))
    return big_label_dict[inp_label]

# def accuracytop3(preds, targs):
# #     pdb.set_trace()
#     preds1 = torch.Tensor(preds)
#     targs1 = torch.LongTensor(targs)
#     preds_3 = preds1.sort(dim=1, descending=True)[1][:, :3]
#     return ((preds_3[:, 0] == targs1) + (preds_3[:, 1] == targs1) +
#             (preds_3[:, 2] == targs1)).float().mean()

def accuracytop3(preds, targs):
    preds_3 = preds.sort(dim=1, descending=True)[1][:, :3]
    return ((preds_3[:, 0] == targs) + (preds_3[:, 1] == targs) +
            (preds_3[:, 2] == targs)).float().mean()

# def accuracytop3(preds, targs):
#     preds_3 = preds.sort(dim=1, descending=True)[1][:, :3]
#     return ((preds_3[:, 0] == targs) + (preds_3[:, 1] == targs) +
#             (preds_3[:, 2] == targs)).float().mean()

def test_acc_csv():
    log_preds, y = learn.TTA(is_test=True)
    probs = np.exp(log_preds)

    probs_1 = np.mean(probs, axis=0)
    # probs_1.shape

    sub_ds = pd.DataFrame(probs_1)
    sub_ds.columns = data.classes
    sub_ds.insert(0, 'id', [o[9:] for o in data.test_ds.fnames])

    SUBM = f'{PATH}sub/'
    os.makedirs(SUBM, exist_ok=True)
    sub = 'id,predicted\n'
    str_format = '{},{} {} {}\n'
    ds_class_array = np.array(data.classes)
    for i in range(len(sub_ds)):
        r1 = sub_ds.iloc[i]
        r11 = np.array(r1[1:])
        r12 = -r11
        r2 = np.argsort(r12)
        r3 = np.array(data.classes)[r2]
        pred_test_fin = r3[:3]
        sub += str_format.format(data.test_ds.fnames[i][9:],
                                 pred_test_fin[0], pred_test_fin[1], pred_test_fin[2])

    with open('sub_fin1.csv', 'w') as f:
        f.write(sub)


In [2]:
mdd = se_resnext101_32x4d()

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext101_32x4d-3b2fe3d8.pth" to /home/arka/.torch/models/se_resnext101_32x4d-3b2fe3d8.pth
100%|██████████| 196466866/196466866 [00:11<00:00, 17394214.12it/s]


In [3]:
mdd

SENet(
  (layer0): Sequential(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (relu1): ReLU(inplace)
    (pool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=True)
  )
  (layer1): Sequential(
    (0): SEResNeXtBottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (se_module): SEModule(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc1): Conv2d(256, 16, kernel_size=(1, 1), stride=(1, 

In [16]:
md3 = nn.Sequential(*children(md), nn.ReLU(), nn.Linear(1000, 211), nn.LogSoftmax())

In [12]:
md2

new_senet(
  (md): SENet(
    (layer0): Sequential(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu1): ReLU(inplace)
      (pool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=True)
    )
    (layer1): Sequential(
      (0): SEResNeXtBottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
        (relu): ReLU(inplace)
        (se_module): SEModule(
          (avg_pool): AdaptiveAvgPool2d(output_size=1)
         

In [20]:
data = get_data(224, 5)

In [17]:
learn1 = ConvLearner.from_model_data(md3, data)

In [18]:
learn1.model

Sequential(
  (0): Sequential(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (relu1): ReLU(inplace)
    (pool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=True)
  )
  (1): Sequential(
    (0): SEResNeXtBottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (se_module): SEModule(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc1): Conv2d(256, 16, kernel_size=(1, 1), stride=(1, 1))
 

In [19]:
learn1.fit(1e-2, 1, cycle_len=1, best_save_name='a', metrics=[accuracy, accuracytop3])

  0%|          | 0/2814 [00:00<?, ?it/s]


RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/THC/generic/THCStorage.cu:58

In [8]:
learn.unfreeze()

In [9]:
learn.fit(1e-2, 1, cycle_len=1, best_save_name='a', metrics=[accuracy, accuracytop3])

  0%|          | 0/2814 [00:00<?, ?it/s]


RuntimeError: size mismatch at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/THC/generic/THCTensorMathBlas.cu:247

In [4]:
# import pdb
class new_senet(nn.Module):
    def __init__(self, md, nout):
        super().__init__()
        self.md = md
        self.lin2 = nn.Linear(1000, nout)
    def forward(self, inp):
        out1 = F.relu(self.md(inp))
        out2 = self.lin2(out1)
        return F.log_softmax(out2, dim=-1)

In [5]:
md2 = new_senet(md, 211)

In [ ]:
md3

In [6]:
learn1 = ConvLearner.from_model_data(md2, data)

learn1.unfreeze()
learn1.fit(1e-2, 1, cycle_len=1, best_save_name='a', metrics=[accuracy, accuracytop3])

  0%|          | 9/2814 [00:03<17:16,  2.71it/s, loss=5.35]


KeyboardInterrupt: 

In [ ]:
learn.unfreeze()
learn.save_fc1()
learn.freeze()
learn.precompute = True

In [ ]:
learn.fit(1e-3, 1, cycle_len=1, best_save_name='a1', metrics=[accuracy, accuracytop3])

In [ ]:
learn.fit([1e-2, 1e-2, 1e-2], 1, 1, best_save_name='a1', metrics=[accuracy, accuracytop3])